In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from sklearn.datasets import fetch_california_housing

In [3]:
housing = fetch_california_housing()

In [4]:
housing.data.shape

(20640, 8)

In [5]:
housing.target.shape

(20640,)

In [6]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

In [7]:
scalled_housing = ss.fit_transform(housing.data)

In [8]:
scalled_housing_with_bias = np.c_[np.ones((housing.data.shape[0])), scalled_housing]

In [9]:
num_examples, num_features = scalled_housing_with_bias.shape

In [10]:
x = tf.constant(scalled_housing_with_bias, dtype=tf.float32)
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32)

In [11]:
theta = tf.Variable(tf.random_normal(shape=(num_features, 1)), dtype=tf.float32)

In [12]:
y_pred = tf.matmul(x, theta)

In [13]:
error = y_pred - y

In [14]:
mse = tf.reduce_mean(tf.square(error))

In [15]:
#optimizer = tf.train.MomentumOptimizer(learning_rate=0.1, momentum=0.9)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0005)
training_op = optimizer.minimize(mse)

In [16]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [17]:
init = tf.global_variables_initializer()

In [18]:
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = 'logs'
logdir = "{}/run-{}/".format(root_logdir, now)

In [19]:
mse_summary = tf.summary.scalar("MSE", mse)
file_wreiter = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [20]:
with tf.Session(config=config) as sess:
    sess.run(init)
    for epoch in range(5000):
        if epoch % 100 == 0:
            summary_str = mse_summary.eval()
            step = epoch
            file_wreiter.add_summary(summary_str, step)
        sess.run(training_op)
        
    file_wreiter.close()
    best_weights = theta.eval()
    print("Best Weights\n", best_weights)

Best Weights
 [[ 2.06630492]
 [ 0.88215923]
 [ 0.30289686]
 [-0.03370323]
 [-0.03664384]
 [ 0.05866125]
 [-0.06255856]
 [ 0.55697358]
 [ 0.59660661]]


In [21]:
A = tf.placeholder(dtype=tf.float32, shape=(None, 3))
B = A + 5

with tf.Session() as sess:
    B1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B2 = B.eval(feed_dict={A: [[1, 2, 3], [4, 5, 6]]})

print(B1)
print(B2)

[[ 6.  7.  8.]]
[[  6.   7.   8.]
 [  9.  10.  11.]]
